<a href="https://colab.research.google.com/github/Pravallikapatnana/FMML_LABS_ASSESSMENTS/blob/main/module_01_lab_02_mlpractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [ ]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [ ]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [ ]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [ ]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [ ]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [ ]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


In [ ]:
Yes, averaging the validation accuracy across multiple splits (e.g., using cross-validation) can often give more consistent and reliable results compared to relying on a single validation split. Cross-validation is a technique used in machine learning to assess the performance of a model and reduce the risk of overfitting by repeatedly splitting the dataset into training and validation subsets.

Here's why averaging validation accuracy across multiple splits can be beneficial for consistency:

1. **Reduced Variance:** A single random split of data into training and validation sets may lead to variability in the performance metric (e.g., accuracy) due to the randomness in the data partitioning. Averaging over multiple splits helps reduce this variance, providing a more stable estimate of model performance.

2. **Better Generalization:** Cross-validation assesses a model's ability to generalize to different subsets of the data. Averaging results across multiple splits gives you a more robust estimate of how well your model is likely to perform on unseen data.

3. **Detecting Overfitting:** If your model performs exceptionally well on a single validation split but poorly on others, it may be overfitting to the specific training-validation split. Averaging over multiple splits helps detect such situations and provides a more accurate assessment of model generalization.

4. **More Reliable Hyperparameter Tuning:** When tuning hyperparameters, cross-validation allows you to evaluate a model's performance across a range of hyperparameter values. Averaging results over multiple splits helps you choose the hyperparameters that consistently lead to better performance.

In summary, averaging validation accuracy across multiple splits, as done in cross-validation, is a best practice in machine learning for obtaining more consistent and trustworthy performance estimates for your models.

In [ ]:
Averaging validation accuracy across multiple splits using techniques like cross-validation provides a more accurate estimate of a model's generalization performance compared to a single validation split. However, it's important to clarify that the "test accuracy" typically refers to the performance of your model on a completely independent and unseen dataset, which is distinct from the validation accuracy obtained through cross-validation.

Here's how it works:

1. **Validation Accuracy (Cross-Validation):** Cross-validation involves splitting your dataset into multiple subsets (folds), training and evaluating your model on each fold, and then averaging the results. This process provides a more accurate estimate of how well your model is likely to perform on unseen data compared to a single validation split. It helps you assess your model's generalization performance more reliably within your dataset.

2. **Test Accuracy:** The test accuracy is obtained by evaluating your trained model on a separate dataset that the model has never seen during training or validation. This independent test dataset simulates how well your model will perform in real-world scenarios where you encounter new, unseen data. The test accuracy is the most accurate estimate of how well your model generalizes to new, unseen examples.

While cross-validation gives you a better estimate of your model's performance within your dataset and helps you tune hyperparameters and assess model quality, the true measure of how well your model will perform in production is the test accuracy on an independent dataset. Therefore, you should still reserve a portion of your data for this purpose and not use the test data for model development or hyperparameter tuning.

In summary, cross-validation provides a more accurate estimate of how well your model performs within your dataset, but the ultimate measure of real-world performance is the test accuracy on an independent dataset.

In [ ]:
The number of iterations in machine learning, particularly in iterative optimization algorithms like gradient descent, can have an impact on the quality of the estimate or the performance of your model. However, whether you get a better estimate with higher iterations depends on several factors, and there isn't a one-size-fits-all answer. Here are some considerations:

1. **Convergence:** In many cases, increasing the number of iterations can help the optimization algorithm converge to a better solution. For example, in gradient descent, more iterations may allow the algorithm to reach a lower value of the cost function, indicating a better-fitting model. However, there's a diminishing return effect, and at some point, further iterations may not significantly improve the estimate.

2. **Overfitting:** If you train your model for too many iterations, it may start to overfit the training data. Overfitting occurs when a model learns to fit the noise in the data rather than the underlying patterns. This can lead to a model that performs well on the training data but poorly on new, unseen data. Regularization techniques are often used to mitigate overfitting.

3. **Computational Resources:** Training for a larger number of iterations may require more computational resources (e.g., time and memory). You should balance the computational cost with the expected improvement in model performance.

4. **Early Stopping:** In practice, it's common to use a technique called early stopping, where you monitor the performance on a validation set during training. When the validation performance stops improving or starts to degrade, you stop training, even if you haven't reached a predetermined number of iterations. This prevents overfitting and ensures you have a model that generalizes well.

5. **Hyperparameter Tuning:** The number of iterations can also be considered a hyperparameter that needs tuning. You may find that the optimal number of iterations varies depending on the dataset and the specific machine learning algorithm you are using.

In summary, the effect of the number of iterations on the estimate depends on the specific problem, algorithm, and data. While increasing iterations can improve the estimate up to a point, it's crucial to monitor for signs of overfitting and use techniques like early stopping to strike a balance between model performance and computational efficiency. The optimal number of iterations may need to be determined through experimentation and hyperparameter tuning.

In [ ]:
Increasing the number of iterations in a machine learning algorithm is not a direct solution for dealing with a very small training dataset or validation dataset. While increasing the number of iterations may help improve the model's fit to the training data, it does not address the fundamental limitations of having a small dataset.

Here are some reasons why simply increasing iterations is not a sufficient solution:

1. **Overfitting:** If you have a very small training dataset, increasing the number of iterations can make the model fit the noise in the data more closely, leading to overfitting. Overfit models perform well on the training data but generalize poorly to new, unseen data. In such cases, you are likely to observe poor performance on the validation or test dataset.

2. **Data Scarcity:** With a small dataset, there may not be enough diverse examples to capture the underlying patterns in the data. No amount of additional iterations can compensate for the lack of representative data.

3. **Computational Efficiency:** While increasing iterations can improve model fit, it may also increase the computational cost and training time. For small datasets, it's often more efficient to focus on other aspects of model development, such as feature engineering, hyperparameter tuning, or using simpler models.

4. **Validation Data:** When you have a small validation dataset, it becomes challenging to reliably assess your model's performance. A small validation dataset can lead to high variability in evaluation metrics, making it difficult to make informed decisions about your model's quality.

To address the challenges posed by small datasets, consider the following approaches:

1. **Data Augmentation:** If possible, augment your small dataset by creating variations of existing examples. This can help introduce more diversity into the data.

2. **Regularization:** Apply regularization techniques (e.g., L1, L2 regularization) to prevent overfitting when working with small datasets.

3. **Transfer Learning:** If applicable, consider using pre-trained models or transfer learning techniques, which leverage knowledge from larger datasets to improve performance on your specific task.

4. **Use Simpler Models:** Simpler models with fewer parameters may generalize better when data is limited.

5. **Cross-Validation:** Use cross-validation to get a more robust estimate of model performance, even with a small dataset.

In summary, while increasing iterations can play a role in model training, it is not a substitute for addressing the inherent challenges of working with small datasets. Careful consideration of data-related issues and appropriate techniques to mitigate them are essential when dealing with limited data.